# Defining your own classes

## User Defined Types

A **class** is a user-programmed Python type (since Python 2.2!)

It can be defined like:

In [1]:
class Room(object):
    pass

Or:

In [2]:
class Room():
    pass

Or:

In [3]:
class Room:
    pass

What's the difference? Before Python 2.2 a class was distinct from all other Python types, which caused some odd behaviour. To fix this, classes were redefined as user programmed types by extending `object`, e.g., class `room(object)`.

So most Python 2 code will use this syntax as very few people want to use old style python classes. Python 3 has formalised this by removing old-style classes, so they can be defined without extending `object`, or indeed without braces.


Just as with other python types, you use the name of the type as a function to make a variable of that type:

In [4]:
zero = int()
type(zero)

int

In [5]:
myroom = Room()
type(myroom)

__main__.Room

In the jargon, we say that an **object** is an **instance** of a particular **class**.

`__main__` is the name of the scope in which top-level code executes, where we've defined the class `Room`.

Once we have an object with a type of our own devising, we can add properties at will:

In [6]:
myroom.name = "Living"

In [7]:
myroom.name

'Living'

The most common use of a class is to allow us to group data into an object in a way that is 
easier to read and understand than organising data into lists and dictionaries.

In [8]:
myroom.capacity = 3
myroom.occupants = ["James", "Sue"]

## Methods

So far, our class doesn't do much!

We define functions **inside** the definition of a class, in order to give them capabilities, just like the methods on built-in
types.

In [9]:
class Room:
    def overfull(self):
        return len(self.occupants) > self.capacity

In [10]:
myroom = Room()
myroom.capacity = 3
myroom.occupants = ["James", "Sue"]

In [11]:
myroom.overfull()

False

In [12]:
myroom.occupants.append(["Clare"])

In [13]:
myroom.occupants.append(["Bob"])

In [14]:
myroom.overfull()

True

When we write methods, we always write the first function argument as `self`, to refer to the object instance itself,
the argument that goes "before the dot".

This is just a convention for this variable name, not a keyword. You could call it something else if you wanted.

## Constructors

Normally, though, we don't want to add data to the class attributes on the fly like that. 
Instead, we define a **constructor** that converts input data into an object. 

In [15]:
class Room:
    def __init__(self, name, exits, capacity, occupants=[]):
        self.name = name
        self.occupants = occupants  # Note the default argument, occupants start empty
        self.exits = exits
        self.capacity = capacity

    def overfull(self):
        return len(self.occupants) > self.capacity

In [16]:
living = Room("Living Room", {"north": "garden"}, 3)

In [17]:
living.capacity

3

Methods which begin and end with **two underscores** in their names fulfil special capabilities in Python, such as
constructors.

## Object-oriented design

In building a computer system to model a problem, therefore, we often want to make:

* classes for each *kind of thing* in our system
* methods for each *capability* of that kind
* properties (defined in a constructor) for each *piece of information describing* that kind


For example, the below program might describe our "Maze of Rooms" system:

We define a "Maze" class which can hold rooms:

In [18]:
import random

In [19]:
class Maze:
    def __init__(self, name):
        self.name = name
        self.rooms = {}

    def add_room(self, room):
        # class relation attribute
        room.maze = self  # The Room needs to know which Maze it is a part of

        # add room to maze
        self.rooms[room.name] = room    # str: room

    def occupants(self):
        """Return all Person instances in Maze"""
        return [
            occupant
            for room in self.rooms.values()             # iterate over Room instances
            for occupant in room.occupants.values()     # iterate over Person instances
        ]

    def wander(self):
        """Move all the people in a random direction"""
        for occupant in self.occupants():
            # each Person wanders
            occupant.wander()

    def describe(self):
        """Describes each individual Room"""
        for room in self.rooms.values():
            room.describe()

    def step(self):
        self.describe()
        print("")
        self.wander()
        print("")

    def simulate(self, steps):
        for _ in range(steps):
            self.step()

And a "Room" class with exits, and people:

In [20]:
class Room:
    def __init__(self, name, exits, capacity, maze=None):
        self.name = name
        self.exits = exits  # Should be a dictionary from directions to room names
        self.capacity = capacity
        self.maze = maze

        self.occupants = {}  # Note the default argument, occupants start empty

    def has_space(self):
        return len(self.occupants) < self.capacity

    def available_exits(self):
        return [
            exit
            for exit, target in self.exits.items()  # iterate over exit names (target)
            if self.maze.rooms[target].has_space()  # check if Room instance has room
        ]

    def random_valid_exit(self):
        """Get random direction (exit) of the Room"""
        if not self.available_exits():
            return None
        return random.choice(self.available_exits())

    def destination(self, exit):
        """Get the Room instance of the exit"""
        return self.maze.rooms[self.exits[exit]]

    def add_occupant(self, occupant):
        # class relation attribute
        occupant.room = self  # The person needs to know which room it is in
        
        # add person to room
        self.occupants[occupant.name] = occupant    # str: Person

    def delete_occupant(self, occupant):
        del self.occupants[occupant.name]

    def describe(self):
        if self.occupants:
            print(f"{self.name}: " + " ".join(self.occupants.keys()))

We define a "Person" class for room occupants:

In [21]:
class Person:
    def __init__(self, name, room=None):
        self.name = name
        self.room = None

    def use(self, exit):
        self.room.delete_occupant(self)
        destination = self.room.destination(exit)
        destination.add_occupant(self)
        print(
            "{some} goes {action} to the {where}".format(
                some=self.name, action=exit, where=destination.name
            )
        )

    def wander(self):
        exit = self.room.random_valid_exit()

        # if exit is not None, then available room exists
        if exit:
            self.use(exit)

And we use these classes to define our people, rooms, and their relationships:

In [22]:
# Outer class
house = Maze("My House")

# Middle class & exits embedded
living = Room(
    "livingroom", {"north": "kitchen", "upstairs": "bedroom", "outside": "garden"}, 2
)
kitchen = Room("kitchen", {"south": "livingroom"}, 1)
garden = Room("garden", {"inside": "livingroom"}, 3)
bedroom = Room("bedroom", {"downstairs": "livingroom", "jump": "garden"}, 1)

# Inner class
james = Person("James")
sue = Person("Sue")
bob = Person("Bob")
clare = Person("Clare")

# Link Maze(house) with Rooms
for room in [living, kitchen, garden, bedroom]:
    house.add_room(room)

# Link Room with Person instances
living.add_occupant(james)
garden.add_occupant(sue)
garden.add_occupant(clare)
bedroom.add_occupant(bob)

# Simulate
random.seed(123)
house.simulate(3)

livingroom: James
garden: Sue Clare
bedroom: Bob

James goes north to the kitchen
Sue goes inside to the livingroom
Clare goes inside to the livingroom
Bob goes jump to the garden

livingroom: Sue Clare
kitchen: James
garden: Bob

Sue goes outside to the garden
Clare goes upstairs to the bedroom
James goes south to the livingroom
Bob goes inside to the livingroom

livingroom: James Bob
garden: Sue
bedroom: Clare

James goes outside to the garden
Bob goes outside to the garden
Sue goes inside to the livingroom
Clare goes downstairs to the livingroom



## Alternative object models

There are many choices for how to design programs to do this. Another choice would be to separately define exits as a different class from rooms. This way, 
we can use arrays instead of dictionaries, but we have to first define all our rooms, then define all our exits.

In [23]:
class Maze:
    def __init__(self, name):
        self.name = name
        self.rooms = []

        # Person instances stored in the larger Maze class
        # Each Person has Rooms stored locally
        self.occupants = []

    def add_room(self, name, capacity):
        result = Room(name, capacity)   # no maze attribute
        self.rooms.append(result)

        # Return newly initialized Room instance
        return result

    def add_exit(self, name, source, target, reverse=None):
        # Directions: name and reverse
        # Rooms: source target

        # source to target exit
        source.add_exit(name, target)

        # target to source exit
        # Note: do not reverse when mirroring the exit
        if reverse:
            target.add_exit(reverse, source)

    def add_occupant(self, name, room):
        self.occupants.append(Person(name, room))
        room.occupancy += 1

    def wander(self):
        "Move all the people in a random direction"
        for occupant in self.occupants:
            occupant.wander()

    def describe(self):
        """Describes each individual Person"""
        for occupant in self.occupants:
            occupant.describe()

    def step(self):
        self.describe()
        print("")
        self.wander()
        print("")

    # simulate people moving around in steps
    def simulate(self, steps):
        for _ in range(steps):
            self.step()

In [24]:
class Room:
    def __init__(self, name, capacity):
        self.name = name
        self.capacity = capacity
        self.occupancy = 0
        self.exits = []     # list of exit classes

        # No list of people. Capacity only
        # Room is stored by each Person

    def has_space(self):
        return self.occupancy < self.capacity

    def available_exits(self):
        return [exit for exit in self.exits if exit.valid()]

    def random_valid_exit(self):
        """Select random valid Exit instance"""
        if not self.available_exits():
            return None
        return random.choice(self.available_exits())

    def add_exit(self, name, target):
        self.exits.append(Exit(name, target))

In [25]:
class Person:
    def __init__(self, name, room=None):
        self.name = name
        self.room = room

    def use(self, exit):
        self.room.occupancy -= 1
        destination = exit.target
        destination.occupancy += 1
        self.room = destination
        print(
            "{some} goes {action} to the {where}".format(
                some=self.name, action=exit.name, where=destination.name
            )
        )

    def wander(self):
        exit = self.room.random_valid_exit()

        # if exit is not None, then available room exists
        if exit:
            self.use(exit)

    def describe(self):
        print("{who} is in the {where}".format(who=self.name, where=self.room.name))

In [26]:
class Exit:
    def __init__(self, name, target):
        self.name = name        # Name of direction
        self.target = target    # Room instance

    def valid(self):
        return self.target.has_space()

In [27]:
# Outer class
house = Maze("My New House")

# Middle class and Links to Maze
living = house.add_room("livingroom", 2)
kitchen = house.add_room("kitchen", 1)
garden = house.add_room("garden", 3)
bed = house.add_room("bedroom", 1)

# Add Exits to Rooms (through Outer class)
house.add_exit("north", living, kitchen, "south")
house.add_exit("upstairs", living, bed, "downstairs")
house.add_exit("outside", living, garden, "inside")
house.add_exit("jump", bed, garden)

# Link Rooms with Person instances (through Outer class)
house.add_occupant("James", living)
house.add_occupant("Sue", garden)
house.add_occupant("Clare", garden)
house.add_occupant("Bob", bed)

# Simulate
random.seed(123)
house.simulate(3)

# Iterating over occupants rather than rooms produces different results

James is in the livingroom
Sue is in the garden
Clare is in the garden
Bob is in the bedroom

James goes north to the kitchen
Sue goes inside to the livingroom
Clare goes inside to the livingroom
Bob goes jump to the garden

James is in the kitchen
Sue is in the livingroom
Clare is in the livingroom
Bob is in the garden

Sue goes outside to the garden
Clare goes upstairs to the bedroom
Bob goes inside to the livingroom

James is in the kitchen
Sue is in the garden
Clare is in the bedroom
Bob is in the livingroom

James goes south to the livingroom
Clare goes jump to the garden
Bob goes upstairs to the bedroom



This is a huge topic, about which many books have been written. The differences between these two designs are important, and will have long-term consequences for the project. That is the how we start to think about **software engineering**, as opposed to learning to program, and is an important part of this course.

## Exercise: Your own solution

Compare the two solutions above. Discuss with a partner which you like better, and why. Then, starting from scratch, design your own. What choices did you make that are different from mine?